<a href="https://colab.research.google.com/github/RafaelBNN/if697/blob/main/Projeto_2_IF697.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importando as Bibliotecas

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
!pip install mlflow
import mlflow

     |████████████████████████████████| 14.4 MB 61 kB/s 
     |████████████████████████████████| 146 kB 48.0 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 79 kB 7.1 MB/s 
     |████████████████████████████████| 1.1 MB 50.9 MB/s 
     |████████████████████████████████| 636 kB 49.6 MB/s 
     |████████████████████████████████| 170 kB 50.2 MB/s 
     |████████████████████████████████| 75 kB 4.5 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158172 sha256=0cb7636a1642a8d39643de14f3805c2f3d343b3fad4465a5ff0ef0f325fa729d
  Stored in directory: /root/.cache/pip/wheels/be/5d/0a/9e13f53f4f5dfb67cd8d245bb7cdffe12f135846f491a283e3
  Created wheel for databricks-cli: filename=databricks_cli-0.15.0-py3-none-any.whl size=105260 sha256=38a9e9a206dd06c07468461673d5f25d6bb3eec52b288aa1940ec17a34894088

# 1. Introdução

Este Notebook busca dar continuidade à análise do dataset estudado no primeiro projeto. Lá, exploramos e tratamos um dataset relacionado a competições de powerlifting, sobre o qual foi realizado um pré-processeamento e uma análise descritiva de dados. 

Neste projeto, daremos prosseguimento a isso aplicando a ideia de *predição* dos dados. 

Os problemas de predição aparecem quando queremos descobrir um atributo desconhecido de um registro a partir de outros atributos desse registro. Esses problemas são, principalmente, divididos em duas categorias: problemas de *classificação* e problemas de *regressão*. Na *classificação*, o objetivo é predizer um atributo categórico de um registro, como o sexo de um indivíduo ou a espécie de um animal, por exemplo, enquanto que, na *regressão*, queremos prever um atributo contínuo, como a altura ou o peso de uma pessoa.

A partir do dataset observado, portanto, vamos aplicar o conceito de regressão ao atributo `BodyweightKg` (o peso corporal do competidor). Vamos treinar alguns modelos a partir de variados algoritmos de *machine learning* para realizar a predição dessa coluna e avaliar a performance de cada um desses modelos.

O dataframe que será utilizado na predição é uma amostra do dataset original. Para a previsão, serão utilizadas as colunas `Sex`, o sexo do competidor; `Age`, a idade do competidor; `Best3SquatKg`, a melhor tentativa dentre as 3 no agachamento; `Best3SBenchKg`, a melhor tentativa dentre as 3 no supino; e `Best3DeadliftKg`, a melhor tentativa dentre as 3 no levantamento terra.

#### Importação do Dataframe

In [3]:
df=pd.read_csv('https://raw.githubusercontent.com/RafaelBNN/datasets/main/powerlifting_final.csv')

df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,Name,Sex,Event,Equipment,Age,AgeClass,Division,BodyweightKg,WeightClassKg,Squat1Kg,Squat2Kg,Squat3Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,Best3BenchKg,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Best3DeadliftKg,TotalKg,Place,Wilks,McCulloch,Glossbrenner,IPFPoints,Tested,Country,Federation,Date,MeetCountry,MeetState,MeetName,Best3SquatKg_norm,Best3BenchKg_norm,Best3DeadliftKg_norm,TotalKg_norm,BodyweightKg_norm,IPFPoints_norm,TotalKg_disc,lof_outlier,Age_disc
0,0,692083,Hayden Fulwood,1,SBD,Raw,18.000000,18-19,MR-T3,88.0,90,NaN,NaN,NaN,167.50,NaN,NaN,NaN,122.50,NaN,NaN,NaN,227.50,517.50,1,334.25,354.31,322.27,482.79,Yes,USA,USAPL,2013-04-27,USA,AL,S.E HS State Wars,-0.385417,-0.528924,0.080000,-0.156780,-0.411065,-0.206464,"(507.5, 576.06]",1,"(15.9, 19.6]"
1,1,621012,Jozef Slimák,1,B,Multi-ply,46.000000,45-49,M-M2,82.2,82.5,NaN,NaN,NaN,NaN,165.0,180.0,-190.0,180.00,NaN,NaN,NaN,NaN,180.00,1,120.85,129.06,116.68,571.78,NaN,Slovakia,WPC,2012-11-06,USA,NV,World Championships,NaN,-0.280420,NaN,-0.728814,-0.464106,-0.037484,"(142.5, 202.5]",1,"(45.5, 49.2]"
2,2,164449,Chase Garrett,1,D,Raw,14.000000,13-15,Teenage 13-15,44.0,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.31,70.31,1,83.29,98.28,82.94,212.28,NaN,NaN,SPF,2008-12-06,USA,NaN,Body by George Classic,NaN,NaN,-0.758347,-0.914729,-0.813443,-0.720126,"(19.999, 142.5]",1,"(12.2, 15.9]"
3,3,353419,Edgar Rodea,1,SBD,Single-ply,27.995064,NaN,Boys,92.9,99.7,NaN,NaN,NaN,204.12,NaN,NaN,NaN,129.27,NaN,NaN,NaN,185.97,519.36,1,326.42,326.42,314.34,420.04,Yes,NaN,THSPA,2017-01-28,USA,TX,Santo Meet,-0.232833,-0.499665,-0.141493,-0.153627,-0.366255,-0.325618,"(507.5, 576.06]",1,"(27.0, 30.7]"
4,4,128144,K. Kaufman,1,B,Raw,46.228441,NaN,Open,90.0,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.15,NaN,NaN,NaN,NaN,145.15,6,92.66,92.66,89.30,558.46,NaN,NaN,USPF,1979-12-08,USA,ND,3rd Annual Midwestern Bench Press Championships,NaN,-0.431035,NaN,-0.787881,-0.392775,-0.062777,"(142.5, 202.5]",1,"(45.5, 49.2]"


In [4]:
df.dtypes

Unnamed: 0                int64
Unnamed: 0.1              int64
Name                     object
Sex                       int64
Event                    object
Equipment                object
Age                     float64
AgeClass                 object
Division                 object
BodyweightKg            float64
WeightClassKg            object
Squat1Kg                float64
Squat2Kg                float64
Squat3Kg                float64
Best3SquatKg            float64
Bench1Kg                float64
Bench2Kg                float64
Bench3Kg                float64
Best3BenchKg            float64
Deadlift1Kg             float64
Deadlift2Kg             float64
Deadlift3Kg             float64
Best3DeadliftKg         float64
TotalKg                 float64
Place                    object
Wilks                   float64
McCulloch               float64
Glossbrenner            float64
IPFPoints               float64
Tested                   object
Country                  object
Federati

In [5]:
# As colunas que serao utilizadas
df = df[['Sex','Age','Best3SquatKg','Best3BenchKg','Best3DeadliftKg','BodyweightKg']]

df.head()

,Sex,Age,Best3SquatKg,Best3BenchKg,Best3DeadliftKg,BodyweightKg
0,1,18.000000,167.50,122.50,227.50,88.0
1,1,46.000000,NaN,180.00,NaN,82.2
2,1,14.000000,NaN,NaN,70.31,44.0
3,1,27.995064,204.12,129.27,185.97,92.9
4,1,46.228441,NaN,145.15,NaN,90.0


In [6]:
len(df)

16676

In [7]:
df.isnull().sum(axis = 0)/len(df)

Sex                0.000000
Age                0.000000
Best3SquatKg       0.243404
Best3BenchKg       0.042756
Best3DeadliftKg    0.190094
BodyweightKg       0.005937
dtype: float64

In [8]:
# Para simplificar, vamos executar um dropna sobre o dataframe

df = df.dropna(subset=['Sex','Age','Best3SquatKg','Best3BenchKg','Best3DeadliftKg','BodyweightKg'])

len(df)

12476

In [9]:
# Alem disso, vamos normalizar as colunas que serao usadas, para evitar confusao no futuro

df['Age'] = ((df['Age'] - df['Age'].min()) / (df['Age'].max() - df['Age'].min())*2) -1
df['Best3SquatKg'] = ((df['Best3SquatKg'] - df['Best3SquatKg'].min()) / (df['Best3SquatKg'].max() - df['Best3SquatKg'].min())*2) -1
df['Best3BenchKg'] = ((df['Best3BenchKg'] - df['Best3BenchKg'].min()) / (df['Best3BenchKg'].max() - df['Best3BenchKg'].min())*2) -1
df['Best3DeadliftKg'] = ((df['Best3DeadliftKg'] - df['Best3DeadliftKg'].min()) / (df['Best3DeadliftKg'].max() - df['Best3DeadliftKg'].min())*2) -1


# 2. Conjuntos de treinamento, validação e teste

Ao trabalhar com modelos de *machine learning*, é imprescindível que seja feita a divisão dos dados entre conjunto de treinamento, conjunto de validação e conjunto de teste (*training set*, *validation set* e *test set*). Essa divisão contribui para a avaliação de como o modelo está generalizando os dados a serem previstos e nos ajuda a identificar problemas como *overfitting*.

Aqui, vamos dividir o dataset utilizando a função `train_test_split` da biblioteca `sklearn`.

In [10]:
from sklearn.model_selection import train_test_split

# Os dataframes 'x' contem as colunas que serao utilizadas para realizar a predicao,
# e os dataframes 'y' contem a coluna sobre a qual sera realizada a predicao.

x_df = df.drop('BodyweightKg', axis=1)
y_df = df['BodyweightKg']

# Por padrao, o test set sera formado por 25% dos dados fornecidos, e o training set, 75%.
# Alem disso, o parametro random_state permite que a escolha dos conjuntos de treinamento e teste
# seja a mesma sempre que executarmos esta linha. 
x_train, x_test, y_train, y_test = train_test_split(x_df,y_df,random_state=10) 

In [11]:
x_train.head()

,Sex,Age,Best3SquatKg,Best3BenchKg,Best3DeadliftKg
10825,0,-0.586207,-0.635417,-0.738462,-0.352941
5873,1,-0.710345,-0.041667,0.076923,0.279412
1923,1,-0.451811,-0.125000,-0.123077,-0.058824
12586,1,-0.650272,-0.166708,-0.534031,-0.057176
13777,1,0.006897,-0.034417,-0.296800,0.236294


In [12]:
y_train.head()

10825     77.00
5873      89.95
1923      89.30
12586     53.80
13777    108.64
Name: BodyweightKg, dtype: float64

In [13]:
len(x_train), len(y_train), len(x_test), len(y_test)

(9357, 9357, 3119, 3119)

# 3. Modelos Iniciais

Aqui, empeçaremos a previsão da coluna `BodyweightKg`. Vamos aplicar alguns algoritmos de regressão aos conjuntos de treinamento e teste e avaliar a performance de cada um dos modelos treinados.

## MLFlow

Antes de aplicar os algoritmos, porém, vamos chamar a função `mlflow.sklearn.autolog()`. Ela permite que os modelos sejam registrados automaticamente logo após sua execução utilizando a ferramenta *MLFlow*. Isso permite o rastreamento de como nosso programa está se comportando e como as predições evoluem ao longo do tempo.

In [14]:
mlflow.sklearn.autolog()

## Regressão Linear

O primeiro algoritmo e o mais simples que vamos aplicar vai ser a simples regressão linear. Esse algoritmo de regressão busca encontrar a melhor reta que se ajusta aos dados, ou seja, a reta que minimiza a soma dos resíduos.

In [ ]:
%%time
from sklearn.linear_model import LinearRegression

modelo1 = LinearRegression()
modelo1.fit(x_train,y_train)
y_pred1 = modelo1.predict(x_test)

CPU times: user 18.7 ms, sys: 3.23 ms, total: 21.9 ms
Wall time: 135 ms


In [ ]:
from sklearn.metrics import mean_squared_error

print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred1))

Mean squared error: 305.69


In [ ]:
modelo1.score(x_test,y_test)

0.42925550418604164

## Decision Tree

Os algoritmos de árvore de decisão estão entre os algoritmos com melhor taxa de acurácia x explicabilidade. Ao passo que eles entregam uma acurácia considerável, esses algoritmos também têm uma boa taxa de interpretabilidade, o que significa que suas previsões são de fácil entendimento, diferente dos modelos que usam redes neurais, por exemplo.

Vamos, então, aplicar o algoritmo `DecisionTreeRegressor` e avaliar sua performance.

In [ ]:
%%time
from sklearn import tree

modelo2 = tree.DecisionTreeRegressor() # se o atributo max_depth nao for especificado, por padrao a profundidade sera a maxima possivel, resultando no possivel overfitting
modelo2.fit(x_train,y_train)
y_pred2 = modelo2.predict(x_test)

CPU times: user 88.5 ms, sys: 82.3 ms, total: 171 ms
Wall time: 124 ms


In [ ]:
modelo2.score(x_test,y_test) # overfitting

-0.13361341267618543

In [ ]:
%%time

# Aqui podemos alterar o atributo max_depth e o atributo min_split_samples (default=2), resultando num score diferente
modelo22 = tree.DecisionTreeRegressor(max_depth=5, random_state=10) 

modelo22.fit(x_train,y_train)
y_pred22 = modelo22.predict(x_test)

CPU times: user 13.4 ms, sys: 364 µs, total: 13.8 ms
Wall time: 13.9 ms


In [ ]:
modelo22.score(x_test,y_test)

0.4250097031452772

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(modelo2, x_train, y_train, cv=10) # modelo com overfit

array([-0.20688602, -0.23683985, -0.16078281, -0.30371535, -0.1631512 ,
       -0.14052197, -0.19057281, -0.31572114, -0.12932309, -0.11248548])

In [ ]:
cross_val_score(modelo22, x_train, y_train, cv=10) # modelo com max_depth=5

array([0.41979619, 0.4097474 , 0.40820003, 0.36196559, 0.41220871,
       0.39218524, 0.40878043, 0.37763231, 0.41366703, 0.42547843])

## Random Forest

O método *Random Forest* é um tipo de algoritmo um tanto parecido com as árvores de decisão. Nele, são geradas múltiplas ávores de decisão que são treinadas e usadas para convergir a uma previsão final. Para treinar cada uma dessas árvores, o algoritmo pode usar partes diferentes dos dados do conjunto de treinamento ou até mesmo diferentes conjuntos de features dentre as que serão utilizadas na previsão.

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor

# O algoritmo de random forest tambem oferece os algortimos de max_depth e min_split_sample, que podem ser alterados para obter-se resultados diferentes
modelo3 = RandomForestRegressor(max_depth=5, random_state=10)
modelo3.fit(x_train,y_train)
y_pred3 = modelo3.predict(x_test)

CPU times: user 882 ms, sys: 5.09 ms, total: 887 ms
Wall time: 899 ms


In [ ]:
modelo3.score(x_test,y_test)

0.44307796297090707

## Redes Neurais

As redes neurais estão entre os mais citados e também mais precisos algoritmos de predição já criados. Seu poder vem do fato de esse tipo de algoritmo ser capaz de encontrar relações das mais altas ordens entre os atributos (diferente, por exemplo, da regressão linear, que é capaz apenas de considerar relações *lineares*). Outra característica que possibilita que as redes neurais sejam tão precisas é a quantidade de hiper-parâmetros, que podem ser alterados dependendo da tarefa de previsão a ser realizada.

Aqui, vamos aplicar o algoritmo `MLPRegressor` aos nossos conjuntos de treinamento e teste e avaliar sua performance com hiper-parâmetros *default*.

In [ ]:
%%time

from sklearn.neural_network import MLPRegressor

modelo4 = MLPRegressor(random_state=10).fit(x_train, y_train)
y_pred4 = modelo4.predict(x_test)

CPU times: user 7.93 s, sys: 6.47 s, total: 14.4 s
Wall time: 7.39 s


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
modelo4.score(x_test,y_test)

0.4352326053108301

# 4. MLFlow

# 5. Avaliação dos Modelos

# 6. Diagnóstico e Aprimoramento